In [1]:

import nbimporter

# from Database_connection.database_connection import connect_db
import requests
import pandas as pd
import numpy as np
from datetime import datetime
from sqlalchemy import MetaData, create_engine

In [2]:
# Temporal db Connection (Fix Route)
def connect_db(database = 'coins'):
    user='root'
    password='240699'
    host='localhost'
    database = database
    

    # Connect to the database
    engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")
    try:
        connection = engine.connect()
        return connection
    except Exception as e:
        raise e

## Historic TVL

Date & Value

In [3]:
response = requests.get('https://api.llama.fi/v2/historicalChainTvl')
data = response.json()
df = pd.DataFrame.from_dict(data, orient = 'Columns')
df['date'] = pd.to_datetime(df['date'], unit='s')
timestamp = datetime.now()
df['updated_at'] = timestamp


In [5]:
# Calculate percentage changes for different time intervals
df['1d_change'] = df['tvl'].pct_change() * 100
df['7d_change'] = df['tvl'].pct_change(periods=7) * 100
df['30d_change'] = df['tvl'].pct_change(periods=30) * 100
df['90d_change'] = df['tvl'].pct_change(periods=90) * 100
df['180d_change'] = df['tvl'].pct_change(periods=180) * 100
df['1y_change'] = df['tvl'].pct_change(periods=365) * 100


In [6]:
df

,date,tvl,updated_at,1d_change,7d_change,30d_change,90d_change,180d_change,1y_change
0,2018-06-29,2.054194e+04,2024-01-03 20:20:37.365339,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-06-30,2.061446e+04,2024-01-03 20:20:37.365339,0.353022,NaN,NaN,NaN,NaN,NaN
2,2018-07-01,2.059701e+04,2024-01-03 20:20:37.365339,-0.084654,NaN,NaN,NaN,NaN,NaN
3,2018-07-02,2.052201e+04,2024-01-03 20:20:37.365339,-0.364099,NaN,NaN,NaN,NaN,NaN
4,2018-07-03,2.054986e+04,2024-01-03 20:20:37.365339,0.135686,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2010,2023-12-30,5.280894e+10,2024-01-03 20:20:37.365339,-2.534582,0.745599,13.076644,36.971846,15.922180,36.831512
2011,2023-12-31,5.343913e+10,2024-01-03 20:20:37.365339,1.193335,0.642726,12.344103,35.291370,16.670904,38.869703
2012,2024-01-01,5.325352e+10,2024-01-03 20:20:37.365339,-0.347334,0.639600,11.226637,37.850330,17.085497,40.051118
2013,2024-01-02,5.570251e+10,2024-01-03 20:20:37.365339,4.598735,6.289998,13.853051,45.258475,23.916683,45.423599


In [7]:
with connect_db('protocols_tvl') as connection:
    df.to_sql('historic', con = connection, if_exists='replace')
    print('Historical TVL inserted')

Historical TVL inserted
